In [144]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

#读取数据
train_df = np.load("./data/训练集/train_x.npy")
train_y = np.load("./data/训练集/train_y.npy")
# test_df = np.load("./data/测试集A/test_x_A.npy")
test_df = np.load("./data/测试集B/test_x_B.npy")

In [146]:
#读取特征
#统计性特征
train_feat = pd.read_csv('./data/train_feat.csv')
train_feat['label'] = train_y
test_feat = pd.read_csv('./data/test_feat_B.csv')

#hrv特征
cao_train_feat = pd.read_excel('./cao/feature/feature_train.xlsx')
cao_test_feat = pd.read_excel('./cao/feature/feature_test_hrv_128.xlsx')
cao_train_feat.drop(columns='label',inplace=True)

#列名重置
cao_train_feat.columns = cao_train_feat.columns + '_cao'
cao_test_feat.columns = cao_test_feat.columns + '_cao'

print(train_feat.shape,test_feat.shape)

(37549, 305) (1887, 304)


In [147]:
#样本平衡
train_df = train_feat[train_feat.label==0][:5000]._append(train_feat[train_feat.label!=0]).reset_index(drop=True)
train_y = train_df['label']
train_df = train_df.drop(columns='label')
test_df = test_feat.reset_index(drop=True)

#合并特征
train_df = pd.concat([train_df,cao_train_feat],axis=1)
test_df = pd.concat([test_df,cao_test_feat],axis=1)

train_df.shape,test_df.shape 

((12741, 432), (1887, 432))

In [ ]:
#对抗验证
import xgboost as xgb
import lightgbm as lgb
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

#分离训练集，测试集
train_df['is_test'] = 0
test_df['is_test'] = 1
feat = [col for col in train_df if col not in ['label','is_test']]
print('筛选前 feat_num:',len(feat))


def Adversarial_val(train_df,test_df,feat_cols,lable):
    #合并
    df = pd.concat([train_df,test_df],axis=0,ignore_index=True)
    df = shuffle(df,random_state=16)
    # model = lgb.LGBMClassifier(metric='auc',verbose=-1).fit(df[feat_cols],df[lable])
    model = xgb.XGBClassifier(n_estimators=100,eval_metric= 'auc').fit(df[feat_cols],df[lable])
    #imp
    feat_imp_df = pd.DataFrame({'feature':feat_cols,'imp':np.zeros(len(feat_cols))})
    imp = model.feature_importances_
    feat_imp_df['imp'] = imp
    feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
    feat_imp_df['rank'] = range(feat_imp_df.shape[0])
    #预测
    df_pre = model.predict_proba(df[feat_cols])[:,1]
    auc_score = roc_auc_score(df['is_test'],df_pre)
    # print('AUC:',auc_score)
    return model,df_pre,feat_imp_df,auc_score

#特征筛选
select_feat = []
drop_feat = []

for feat in tqdm(feat):
    adv_model,df_pre,feat_imp_df,auc_score=Adversarial_val(train_df,test_df,[feat],'is_test')
    if auc_score<0.66:
        select_feat.append(feat)
    else:
        drop_feat.append(feat)
print('筛选后 feat_num:',len(select_feat))
feat_cols = select_feat

In [148]:
#对抗验证筛选
drop_feat = ['bo_mean', 'hr_mean', 'bo_shift_1_mean', 'hr_shift_1_mean', 'bo_shift_5_mean','hr_shift_5_mean', 'bo_shift_10_mean', 'hr_shift_10_mean', 'bo_shift_15_mean', 
  'hr_shift_15_mean', 'bo_shift_20_mean', 'hr_shift_20_mean', 'bo_shift_30_mean', 'bo_win10_mean_mean', 'hr_win10_mean_mean', 'hr_win10_std_mean', 'bo_hr_sub_mean', 
  'bo_hr_diff_mean', 'hr_win10_mean_max', 'bo_hr_sub_max', 'hr_win10_mean_min', 'bo_hr_sub_min','hr_std', 'hr_shift_1_std', 'hr_shift_5_std', 'hr_shift_10_std', 'hr_shift_15_std',
  'hr_diff_15_std', 'bo_hr_sub_std', 'bo_win10_mean_median', 'hr_win10_mean_median', 'hr_win10_std_median', 'bo_hr_sub_median', 'hr_var', 'hr_shift_1_var', 'hr_shift_5_var',
  'hr_shift_10_var', 'hr_shift_15_var', 'hr_diff_15_var', 'bo_hr_sub_var', 'bo_diff_1_kurt','bo_diff_5_kurt', 'hr_shift_15_kurt', 'cvsd_bo', 'mean_nni_hr', 'sdnn_hr', 'cvsd_hr', 
  'cvnni_hr', 'mean_hr_hr', 'std_hr_hr', 'mean_nni_hr_60_cao', 'sdnn_hr_60_cao', 'sdsd_hr_60_cao', 'rmssd_hr_60_cao', 'range_nni_hr_60_cao', 'cvnni_hr_60_cao', 'mean_hr_hr_60_cao', 
  'sdnn_index_hr_60_cao', 'hf_hr_60_cao', 'lf_hf_ratio_hr_60_cao', 'lfnu_hr_60_cao', 'hfnu_hr_60_cao', 'total_power_hr_60_cao', 'vlf_hr_60_cao', 'csi_hr_60_cao', 'cvi_hr_60_cao', 
  'Modified_csi_hr_60_cao', 'sd1_hr_60_cao', 'sd2_hr_60_cao', 'ratio_sd2_sd1_hr_60_cao', 'sampen_hr_60_cao', 'mean_nni_spo2_60_cao', 'mean_hr_spo2_60_cao', 'mean_nni_hr_180_cao', 
  'sdnn_hr_180_cao', 'cvnni_hr_180_cao', 'mean_hr_hr_180_cao', 'std_hr_hr_180_cao', 'sdnn_index_hr_180_cao', 'lf_hr_180_cao', 'hf_hr_180_cao', 'vlf_hr_180_cao', 'csi_hr_180_cao', 
  'Modified_csi_hr_180_cao', 'sd2_hr_180_cao', 'ratio_sd2_sd1_hr_180_cao', 'sampen_hr_180_cao', 'mean_nni_spo2_180_cao', 'mean_hr_spo2_180_cao']

#人为筛选
hr_drop_cols = ['mean_nni_hr', 'sdnn_hr', 'sdsd_hr', 'nni_50_hr', 'pnni_50_hr',
       'nni_20_hr', 'pnni_20_hr', 'rmssd_hr', 'median_nni_hr', 'range_nni_hr',
       'cvsd_hr', 'cvnni_hr', 'mean_hr_hr', 'max_hr_hr', 'min_hr_hr',
       'std_hr_hr']
rs_drop_cols = ['breathing_amplitude_hr_180_cao','breathing_frequency_hr_180_cao','snr_hr_180_cao','breathing_frequency_spo2_180_cao','breathing_amplitude_spo2_180_cao','snr_spo2_180_cao']#
feat_cols =[col for col in train_df if col not in ['cvnni_bo', 'nni_50_hr', 'pnni_50_hr']+hr_drop_cols+drop_feat+rs_drop_cols]
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
len(feat_cols)

332

In [149]:
#lgb
import warnings
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report,f1_score,precision_score, recall_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

def my_metrice(preds, eval_data):
    label = eval_data.get_label()
    preds = np.argmax(preds, axis=1)
    score = accuracy_score(label,preds)
    return ('score', score, True) 

def lgb_model(train_x, train_y, test_x):
    seeds=[2024]
    oof = np.zeros([train_x.shape[0], 3])
    test_predict = np.zeros([test_x.shape[0], 3])
    feat_imp_df = pd.DataFrame()
    feat_imp_df['feature'] = train_x.columns
    feat_imp_df['imp'] = 0
    for seed in seeds:
        print('Seed:',seed)
        folds = 12
        kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        acc_scores = []
        # train_x = train_x.values
        # train_y = train_y.values
        
        for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
            print("|  Model  Fold  {}  Training Start           |".format(str(i + 1)))
            
            trn_x, trn_y, val_x, val_y = train_x.values[train_index], train_y[train_index], train_x.values[valid_index], \
                                        train_y[valid_index] 
            
            lgb_params={"boosting_type": "gbdt","objective": "multiclass","metric": "multiclass",
                        'num_class':3,"learning_rate": 0.1,'verbose': -1,'num_leaves':36,
                        }#'colsample_bytree':0.85,'subsample':0.85
            dtrain = lgb.Dataset(trn_x, label=trn_y)
            dval = lgb.Dataset(val_x, label=val_y)
            lgb_model = lgb.train(lgb_params, dtrain, valid_sets=[dval], num_boost_round=20000,callbacks=[lgb.early_stopping(100), lgb.log_evaluation(500)],
                                  feval=my_metrice)

            val_pred  = lgb_model.predict(val_x,num_iteration=lgb_model.best_iteration)
            test_pred = lgb_model.predict(test_x,num_iteration=lgb_model.best_iteration)

            #特征重要性
            feat_imp_df['imp'] += lgb_model.feature_importance(importance_type='gain') / folds/ len(seeds)
            feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
            feat_imp_df['rank'] = range(feat_imp_df.shape[0])
            
            oof[valid_index] = val_pred/ len(seeds)
            test_predict += test_pred / kf.n_splits / len(seeds)
            
            acc_score = accuracy_score(val_y,np.argmax(val_pred,axis=1))
            print(acc_score)
            acc_scores.append(acc_score)
            print('AVG_acc :',sum(acc_scores)/len(acc_scores))
            print('std :',np.std(acc_scores))
        print('oof:',classification_report(train_y,np.argmax(oof,axis=1)))
        
    return oof, test_predict,feat_imp_df

# 训练LGB模型
lgb_oof, lgb_test, lgb_imp_df = lgb_model(train_df[feat_cols], train_y, test_df[feat_cols])

Seed: 2024
|  Model  Fold  1  Training Start           |
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[40]	valid_0's multi_logloss: 0.683096	valid_0's score: 0.696798
0.696798493408663
AVG_acc : 0.696798493408663
std : 0.0
|  Model  Fold  2  Training Start           |
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[38]	valid_0's multi_logloss: 0.641321	valid_0's score: 0.72693
0.7269303201506592
AVG_acc : 0.7118644067796611
std : 0.015065913370998107
|  Model  Fold  3  Training Start           |
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[113]	valid_0's multi_logloss: 0.610935	valid_0's score: 0.736347
0.736346516007533
AVG_acc : 0.7200251098556184
std : 0.016867581723219578
|  Model  Fold  4  Training Start           |
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[81]	valid_0's mu

In [152]:
submission = pd.read_csv('./sub/B/submit_example_B.csv')
submission['label'] = np.argmax(lgb_test,axis=1)
# submission.to_csv('./sub/result.csv', index = False)